<a href="https://colab.research.google.com/github/kunal2002/LegalBrain-VectorSearch/blob/PineconeData/PineconeData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Full** Judgment File

In [228]:
import json
f1 = open('1950-1955.json')
data1 = json.load(f1)
attributes = ['id', 'date', 'tagline','judgment'] # 0 , 1, 2 , 3
# print(len(data[0][attributes[2]]))
full_judgment = []
# // 932 judgments
# //id date tagline judgment
for i in range(len(data1)):
  full_judgment.append(data1[i][attributes[3]])
print(len(full_judgment))
#   print(data[i][attributes[2]])

932


#Summarized Judgment File

In [ ]:
import json
f = open('data1.json')
data = json.load(f)
attributes = ['id', 'date', 'tagline','summarized_judgment'] # 0 , 1, 2 , 3
# print(len(data[0][attributes[2]]))
judgment = []
taglines = []
ids = []
# // 932 judgments
# //id date tagline judgment
for i in range(len(data)):
  judgment.append(data[i][attributes[3]])
  taglines.append(data[i][attributes[2]])
  ids.append(data[i][attributes[0]])
print(len(judgment), len(taglines), len(ids))
#   print(data[i][attributes[2]])

932 932 932


In [ ]:
data[0]['tagline']

'Kapildeo Singh vs The King on 24 January, 1950'

In [ ]:
!pip install -qU pinecone-client openai datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.2/177.2 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.7/283.7 kB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 44.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
pinecone_api_key = "39123716-bb7b-441e-9f02-1a9e7e8bc564"
pinecone_env = "us-west4-gcp-free"

In [ ]:
pip install langchain

In [ ]:
from langchain.vectorstores import Pinecone
from langchain.embeddings.openai import OpenAIEmbeddings
import pinecone

/usr/local/lib/python3.10/dist-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [152]:
index_name = 'vectorsearch'

# initialize connection to pinecone (get API key at app.pinecone.io)
pinecone.init(
    api_key="39123716-bb7b-441e-9f02-1a9e7e8bc564",
    environment="us-west4-gcp-free"  # find next to api key in console
)
# check if 'openai' index already exists (only create index if not)
# if index_name not in pinecone.list_indexes():
#     pinecone.create_index(index_name, dimension=len(embeds[0]))
# connect to index
index = pinecone.Index(index_name)

In [ ]:
MODEL = "text-embedding-ada-002"

In [161]:
import openai
import os
Openai_api_key = "<Your OpenAI API KEY>"
os.environ["OPENAI_API_KEY"] = Openai_api_key
openai_env_key = "<Your OpenAI Env Key>"
openai.api_key = Openai_api_key

# Creating **Embeddings** for summarized judgments and uploading to **Pinecone**

In [162]:
from tqdm.auto import tqdm

count = 0  # we'll use the count to create unique IDs
batch_size = 32  # process everything in batches of 32
for i in tqdm(range(0, len(data), batch_size)):
  judge = []
  # set end position of batch
  i_end = min(i+batch_size, len(data))
  # get batch of lines and IDs
  # tagline + summarized judgment zipped together
  for j in range(i, i_end):
    judge.append(taglines[j] + "\n" + judgment[j] + "\n")
  lines_batch = judge
  print(len(lines_batch), end = " ")
  ids_batch = [str(ids[n]) for n in range(i, i_end)]
  # create embeddings
  # print(ids_batch)
  res = openai.Embedding.create(input=lines_batch, engine=MODEL, )
  embeds = [record['embedding'] for record in res['data']]
  meta = [{'text' : line} for line in lines_batch]
  # print(meta)
  to_upsert = zip(ids_batch,embeds, meta)
  index.upsert(vectors=list(to_upsert))
# # print(to_upsert)
# # # upsert to Pinecone
# index.upsert(vectors=list(to_upsert))

  0%|          | 0/30 [00:00<?, ?it/s]

32 32 32 32 32 32 32 32 32 32 32 32 32 32 32 32 32 32 32 32 32 32 32 32 32 32 32 32 32 4 

# Using OpenAI to answer questions about the **Case/Judgment**

Not working good on summarized judgments due to poor quality of summarization

In [163]:
from langchain.llms import OpenAI
from langchain.chains.question_answering import load_qa_chain

In [164]:
llm = OpenAI(temperature = 0, openai_api_key= Openai_api_key)
chain = load_qa_chain(llm, chain_type = "stuff")

In [190]:
embeddings = OpenAIEmbeddings()
docsearch = Pinecone.from_existing_index(index_name, embeddings)
docs = docsearch.similarity_search(query)

In [209]:
query = "Who is Bhikaji Narain Dhakras?"
# query1 = "Where did Kapildeo singh committed the crime?"

In [212]:
chain.run(input_documents = docs, question = query3)

" I don't know."

#Querying Pinecone by making embeddings using openai

In [231]:
query = "Tell me about kapildeo singh?"
query1 = "Who is Bhikaji Narain Dhakras?"
query3 = "Tell me about the case of AK Gopalan Vs the State of Madras?"
query4 = "What is doctrine of severibility?"
query5 = "Tell me about State of Madras vs Champakam dorairajan?"
query6 = "Shankari Prasad singh deo vs Union of India"
xq = openai.Embedding.create(input=query6, engine=MODEL)['data'][0]['embedding']
res = index.query([xq], top_k = 5, include_metadata=True)
for match in res['matches']:
    print(f"{match['score']:.2f}: {match['metadata']['text']}")

0.88: Sri Sankari Prasad Singh Deo vs Union Of India And State Of ... on 5 October, 1951
sri sankari prasad singh deo vs. respondent : india and state of bihar are not ultra vires or unconstitutional. the constitution of india is competent to exercise the power of amending the constitution under art. in the constitution, the constitution ( first amendment ) act, 1951, which has inserted, inter alia, arts. 31a and 3lb in india are not unconstitutional.

0.87: Sri Sri Sri Kishore Chandra Singh ... vs Babu Ganesh Prasad Bhagat And ... on 9 March, 1954
sri sri sri kishore chandra singh deo and srishore chandra chandra singh are among those to be involved in the case - legal effect of decision under s. 33 ( 1 ) proviso of the act requires that a person eats, drinks and sleeps at that place and it is not necessary that he should own it.

0.87: Rao Shiva Bahadur Singh vs The State Of Vindhya Pradesh And ... on 7 April, 1955
judgment das, j. 1, is a petition for a writ in the nature of a writ.

#For Fetching the real judgment

In [232]:
index_of_judgment = ids.index(res['matches'][0]['id'])
full_judgment[index_of_judgment]

'           PETITIONER:\nSRI SANKARI PRASAD SINGH DEO\n\n\tVs.\n\nRESPONDENT:\nUNION OF INDIA AND STATE OF BIHAR(And Other Cases).\n\nDATE OF JUDGMENT:\n05/10/1951\n\nBENCH:\nSASTRI, M. PATANJALI\nBENCH:\nSASTRI, M. PATANJALI\nKANIA, HIRALAL J. (CJ)\nMUKHERJEA, B.K.\nDAS, SUDHI RANJAN\nAIYAR, N. CHANDRASEKHARA\n\nCITATION:\n 1951 AIR  458\t\t  1952 SCR   89\n CITATOR INFO :\n F\t    1952 SC 252\t (1,30)\n RF\t    1954 SC 257\t (4)\n R\t    1959 SC 395\t (28)\n E&D\t    1959 SC 512\t (4)\n F\t    1965 SC 845\t (20,21,23,24,25,27,33,35,38,39\n R\t    1965 SC1636\t (25)\n O\t    1967 SC1643\t (12,14,23,27,43,44,56,59,61,63\n RF\t    1973 SC1461\t (16,20,27,30,32,38,39,44,46,88\n RF\t    1975 SC1193\t (17)\n RF\t    1975 SC2299\t (649)\n RF\t    1980 SC1789\t (96)\n RF\t    1980 SC2056\t (61)\n RF\t    1980 SC2097\t (6)\n D\t    1981 SC 271\t (19,33,42,43)\n RF\t    1986 SC1272\t (78)\n RF\t    1986 SC1571\t (34)\n RF\t    1987 SC1140\t (3)\n\n\nACT:\n    Constitution  (First  Amendment) A